In [1]:
from datasets import Dataset, DatasetDict
from tqdm import tqdm
import pandas as pd
import json
import wandb
import os

/home/kyobrien/conda/envs/eval-aug/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
project_name = "healthyml/In-Context Domain Transfer Improves Out-of-Domain Robustness"
table_paths = {
    # BOSS Sentiment
    # ID
    # BERT
    "BOSS_Sentiment_ID_BERT_Insert": f"",
    "BOSS_Sentiment_ID_BERT_Substitute": f"",
    "BOSS_Sentiment_ID_BERT_Translate": f"",
    "BOSS_Sentiment_ID_BERT_Paraphrase": f"",
    "BOSS_Sentiment_ID_BERT_ICR": "run-yanx1dus-validation_stabilityaiStableBeluga7b_random_16_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",

    # T5
    "BOSS_Sentiment_ID_T5_Insert": f"",
    "BOSS_Sentiment_ID_T5_Substitute": f"",
    "BOSS_Sentiment_ID_T5_Translate": f"",
    "BOSS_Sentiment_ID_T5_Paraphrase": f"",
    "BOSS_Sentiment_ID_T5_ICR": f"",

    # Falcon
    "BOSS_Sentiment_ID_Falcon_Insert": f"",
    "BOSS_Sentiment_ID_Falcon_Substitute": f"",
    "BOSS_Sentiment_ID_Falcon_Translate": f"",
    "BOSS_Sentiment_ID_Falcon_Paraphrase": f"",
    "BOSS_Sentiment_ID_Falcon_ICR": f"",

    # SST-5
    # BERT
    "BOSS_Sentiment_SST-5_BERT_Insert": f"",
    "BOSS_Sentiment_SST-5_BERT_Substitute": f"",
    "BOSS_Sentiment_SST-5_BERT_Translate": f"",
    "BOSS_Sentiment_SST-5_BERT_Paraphrase": f"",
    "BOSS_Sentiment_SST-5_BERT_ICR": f"",

    # T5
    "BOSS_Sentiment_SST-5_T5_Insert": f"",
    "BOSS_Sentiment_SST-5_T5_Substitute": f"",
    "BOSS_Sentiment_SST-5_T5_Translate": f"",
    "BOSS_Sentiment_SST-5_T5_Paraphrase": f"",
    "BOSS_Sentiment_SST-5_T5_ICR": f"",

    # Falcon
    "BOSS_Sentiment_SST-5_Falcon_Insert": f"",
    "BOSS_Sentiment_SST-5_Falcon_Substitute": f"",
    "BOSS_Sentiment_SST-5_Falcon_Translate": f"",
    "BOSS_Sentiment_SST-5_Falcon_Paraphrase": f"",
    "BOSS_Sentiment_SST-5_Falcon_ICR": f"",

    # SemEval
    # BERT
    "BOSS_Sentiment_SemEval_BERT_Insert": f"",
    "BOSS_Sentiment_SemEval_BERT_Substitute": f"",
    "BOSS_Sentiment_SemEval_BERT_Translate": f"",
    "BOSS_Sentiment_SemEval_BERT_Paraphrase": f"",
    "BOSS_Sentiment_SemEval_BERT_ICR": f"",

    # T5
    "BOSS_Sentiment_SemEval_T5_Insert": f"",
    "BOSS_Sentiment_SemEval_T5_Substitute": f"",
    "BOSS_Sentiment_SemEval_T5_Translate": f"",
    "BOSS_Sentiment_SemEval_T5_Paraphrase": f"",
    "BOSS_Sentiment_SemEval_T5_ICR": f"",

    # Falcon
    "BOSS_Sentiment_SemEval_Falcon_Insert": f"",
    "BOSS_Sentiment_SemEval_Falcon_Substitute": f"",
    "BOSS_Sentiment_SemEval_Falcon_Translate": f"",
    "BOSS_Sentiment_SemEval_Falcon_Paraphrase": f"",
    "BOSS_Sentiment_SemEval_Falcon_ICR": f"",

    # Dynasent
    # BERT
    "BOSS_Sentiment_Dynasent_BERT_Insert": f"",
    "BOSS_Sentiment_Dynasent_BERT_Substitute": f"",
    "BOSS_Sentiment_Dynasent_BERT_Translate": f"",
    "BOSS_Sentiment_Dynasent_BERT_Paraphrase": f"",
    "BOSS_Sentiment_Dynasent_BERT_ICR": f"",

    # T5
    "BOSS_Sentiment_Dynasent_T5_Insert": f"",
    "BOSS_Sentiment_Dynasent_T5_Substitute": f"",
    "BOSS_Sentiment_Dynasent_T5_Translate": f"",
    "BOSS_Sentiment_Dynasent_T5_Paraphrase": f"",
    "BOSS_Sentiment_Dynasent_T5_ICR": f"",

    # Falcon
    "BOSS_Sentiment_Dynasent_Falcon_Insert": f"",
    "BOSS_Sentiment_Dynasent_Falcon_Substitute": f"",
    "BOSS_Sentiment_Dynasent_Falcon_Translate": f"",
    "BOSS_Sentiment_Dynasent_Falcon_Paraphrase": f"",
    "BOSS_Sentiment_Dynasent_Falcon_ICR": f"",

    # BOSS Toxicity
    # ID
    # BERT
    "BOSS_Toxicity_ID_BERT_Insert": f"",
    "BOSS_Toxicity_ID_BERT_Substitute": f"",
    "BOSS_Toxicity_ID_BERT_Translate": f"",
    "BOSS_Toxicity_ID_BERT_Paraphrase": f"",
    "BOSS_Toxicity_ID_BERT_ICR": f"",

    # T5
    "BOSS_Toxicity_ID_T5_Insert": f"",
    "BOSS_Toxicity_ID_T5_Substitute": f"",
    "BOSS_Toxicity_ID_T5_Translate": f"",
    "BOSS_Toxicity_ID_T5_Paraphrase": f"",
    "BOSS_Toxicity_ID_T5_ICR": f"",

    # Falcon
    "BOSS_Toxicity_ID_Falcon_Insert": f"",
    "BOSS_Toxicity_ID_Falcon_Substitute": f"",
    "BOSS_Toxicity_ID_Falcon_Translate": f"",
    "BOSS_Toxicity_ID_Falcon_Paraphrase": f"",
    "BOSS_Toxicity_ID_Falcon_ICR": f"",

    # Toxigen
    # BERT
    "BOSS_Toxicity_Toxigen_BERT_Insert": f"",
    "BOSS_Toxicity_Toxigen_BERT_Substitute": f"",
    "BOSS_Toxicity_Toxigen_BERT_Translate": f"",
    "BOSS_Toxicity_Toxigen_BERT_Paraphrase": f"",
    "BOSS_Toxicity_Toxigen_BERT_ICR": f"",

    # T5
    "BOSS_Toxicity_Toxigen_T5_Insert": f"",
    "BOSS_Toxicity_Toxigen_T5_Substitute": f"",
    "BOSS_Toxicity_Toxigen_T5_Translate": f"",
    "BOSS_Toxicity_Toxigen_T5_Paraphrase": f"",
    "BOSS_Toxicity_Toxigen_T5_ICR": f"",

    # Falcon
    "BOSS_Toxicity_Toxigen_Falcon_Insert": f"",
    "BOSS_Toxicity_Toxigen_Falcon_Substitute": f"",
    "BOSS_Toxicity_Toxigen_Falcon_Translate": f"",
    "BOSS_Toxicity_Toxigen_Falcon_Paraphrase": f"",
    "BOSS_Toxicity_Toxigen_Falcon_ICR": f"",

    # ImplicitHate
    # BERT
    "BOSS_Toxicity_ImplicitHate_BERT_Insert": f"",
    "BOSS_Toxicity_ImplicitHate_BERT_Substitute": f"",
    "BOSS_Toxicity_ImplicitHate_BERT_Translate": f"",
    "BOSS_Toxicity_ImplicitHate_BERT_Paraphrase": f"",
    "BOSS_Toxicity_ImplicitHate_BERT_ICR": f"",

    # T5
    "BOSS_Toxicity_ImplicitHate_T5_Insert": f"",
    "BOSS_Toxicity_ImplicitHate_T5_Substitute": f"",
    "BOSS_Toxicity_ImplicitHate_T5_Translate": f"",
    "BOSS_Toxicity_ImplicitHate_T5_Paraphrase": f"",
    "BOSS_Toxicity_ImplicitHate_T5_ICR": f"",

    # Falcon
    "BOSS_Toxicity_ImplicitHate_Falcon_Insert": f"",
    "BOSS_Toxicity_ImplicitHate_Falcon_Substitute": f"",
    "BOSS_Toxicity_ImplicitHate_Falcon_Translate": f"",
    "BOSS_Toxicity_ImplicitHate_Falcon_Paraphrase": f"",
    "BOSS_Toxicity_ImplicitHate_Falcon_ICR": f"",

    # AdvCivil
    # BERT
    "BOSS_Toxicity_AdvCivil_BERT_Insert": f"",
    "BOSS_Toxicity_AdvCivil_BERT_Substitute": f"",
    "BOSS_Toxicity_AdvCivil_BERT_Translate": f"",
    "BOSS_Toxicity_AdvCivil_BERT_Paraphrase": f"",
    "BOSS_Toxicity_AdvCivil_BERT_ICR": f"",

    # T5
    "BOSS_Toxicity_AdvCivil_T5_Insert": f"",
    "BOSS_Toxicity_AdvCivil_T5_Substitute": f"",
    "BOSS_Toxicity_AdvCivil_T5_Translate": f"",
    "BOSS_Toxicity_AdvCivil_T5_Paraphrase": f"",
    "BOSS_Toxicity_AdvCivil_T5_ICR": f"",

    # Falcon
    "BOSS_Toxicity_AdvCivil_Falcon_Insert": f"",
    "BOSS_Toxicity_AdvCivil_Falcon_Substitute": f"",
    "BOSS_Toxicity_AdvCivil_Falcon_Translate": f"",
    "BOSS_Toxicity_AdvCivil_Falcon_Paraphrase": f"",
    "BOSS_Toxicity_AdvCivil_Falcon_ICR": f"",

    # AG News Tweets
    # ID
    # BERT
    "AgNewsTweets_ID_BERT_Insert": f"",
    "AgNewsTweets_ID_BERT_Substitute": f"",
    "AgNewsTweets_ID_BERT_Translate": f"",
    "AgNewsTweets_ID_BERT_Paraphrase": f"",
    "AgNewsTweets_ID_BERT_ICR": f"",

    # T5
    "AgNewsTweets_ID_T5_Insert": f"",
    "AgNewsTweets_ID_T5_Substitute": f"",
    "AgNewsTweets_ID_T5_Translate": f"",
    "AgNewsTweets_ID_T5_Paraphrase": f"",
    "AgNewsTweets_ID_T5_ICR": f"",

    # Falcon
    "AgNewsTweets_ID_Falcon_Insert": f"",
    "AgNewsTweets_ID_Falcon_Substitute": f"",
    "AgNewsTweets_ID_Falcon_Translate": f"",
    "AgNewsTweets_ID_Falcon_Paraphrase": f"",
    "AgNewsTweets_ID_Falcon_ICR": f"",

    # Tweets
    # BERT
    "AgNewsTweets_Tweets_BERT_Insert": f"",
    "AgNewsTweets_Tweets_BERT_Substitute": f"",
    "AgNewsTweets_Tweets_BERT_Translate": f"",
    "AgNewsTweets_Tweets_BERT_Paraphrase": f"",
    "AgNewsTweets_Tweets_BERT_ICR": f"",

    # T5
    "AgNewsTweets_Tweets_T5_Insert": f"",
    "AgNewsTweets_Tweets_T5_Substitute": f"",
    "AgNewsTweets_Tweets_T5_Translate": f"",
    "AgNewsTweets_Tweets_T5_Paraphrase": f"",
    "AgNewsTweets_Tweets_T5_ICR": f"",

    # Falcon
    "AgNewsTweets_Tweets_Falcon_Insert": f"",
    "AgNewsTweets_Tweets_Falcon_Substitute": f"",
    "AgNewsTweets_Tweets_Falcon_Translate": f"",
    "AgNewsTweets_Tweets_Falcon_Paraphrase": f"",
    "AgNewsTweets_Tweets_Falcon_ICR": f"",
}

print(json.dumps(table_paths, indent=4))

{
    "BOSS_Sentiment_ID_BERT_Insert": "",
    "BOSS_Sentiment_ID_BERT_Substitute": "",
    "BOSS_Sentiment_ID_BERT_Translate": "",
    "BOSS_Sentiment_ID_BERT_Paraphrase": "",
    "BOSS_Sentiment_ID_BERT_ICR": "run-yanx1dus-validation_stabilityaiStableBeluga7b_random_16_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_ID_T5_Insert": "",
    "BOSS_Sentiment_ID_T5_Substitute": "",
    "BOSS_Sentiment_ID_T5_Translate": "",
    "BOSS_Sentiment_ID_T5_Paraphrase": "",
    "BOSS_Sentiment_ID_T5_ICR": "",
    "BOSS_Sentiment_ID_Falcon_Insert": "",
    "BOSS_Sentiment_ID_Falcon_Substitute": "",
    "BOSS_Sentiment_ID_Falcon_Translate": "",
    "BOSS_Sentiment_ID_Falcon_Paraphrase": "",
    "BOSS_Sentiment_ID_Falcon_ICR": "",
    "BOSS_Sentiment_SST-5_BERT_Insert": "",
    "BOSS_Sentiment_SST-5_BERT_Substitute": "",
    "BOSS_Sentiment_SST-5_BERT_Translate": "",
    "BOSS_Sentiment_SST-5_BERT_Paraphrase": "",
    "BOSS_Sentiment_SST-5_BERT_ICR": "",
    "BOSS_Sentiment_S

In [17]:
run = wandb.init()

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


In [22]:
# for each table path, run.use_artifact and download the artifact.
local_paths = {}
missing_tables = []
for table_name, table_path in tqdm(table_paths.items()):
    if table_path == "":
        missing_tables.append(table_name)
        continue

    download_path = f"{project_name}/{table_path}"
    print(f"Downloading {table_name} --- {download_path}")
    artifact = run.use_artifact(download_path, type="run_table")
    artifact_dir = artifact.download()
    file_name = os.listdir(artifact_dir)[0]
    local_paths[table_name] = f"{artifact_dir}/{file_name}"

print(f"\nMissing tables: {missing_tables}")
print(json.dumps(local_paths, indent=4))

  0%|          | 0/150 [00:00<?, ?it/s]

  3%|▎         | 4/150 [03:18<2:00:39, 49.58s/it]


KeyboardInterrupt: 

In [25]:
column_renaming = {
    "entropy": "tta_prediction_entropy",
    "mean probs": "tta_mean_class_probs",
    "all probs": "tta_all_class_probs",
    "all entropies": "tta_all_class_entropy",
    "latency": "tta_inference_latency",
    "input": "augmentations",
    "original_input": "original_text",
    "judgment": "tta_predicted_class",
    "original judgment": "original_predicted_class",
    "original entropy": "original_prediction_entropy",
    "entropy decrease": "prediction_entropy_decrease",
    "entropy decreased": "prediction_entropy_decreased",
}
columns_to_drop = ["style prompt", "mean exemplar distance", "prompt"]
columns_order = ["outcome", "original_text", "augmentations", "original_predicted_class", "tta_predicted_class", "label", "tta_inference_latency", "original_prediction_entropy", "tta_prediction_entropy", "prediction_entropy_decreased", "prediction_entropy_decrease", "tta_mean_class_probs", "tta_all_class_probs", "tta_all_class_entropy", ]

In [30]:
test_dict = DatasetDict()
path = "/home/kyobrien/repos/In-Context-Domain-Transfer-Improves-Out-of-Domain-Robustness/datasets/analysis/export_logs/artifacts/run-yanx1dus-validation_stabilityaiStableBeluga7b_random_16_Kyle1668bosssentimentbertbaseuncased_style_logs:v0/validation_stabilityai/StableBeluga-7b_random_16_Kyle1668/boss-sentiment-bert-base-uncased_style_logs.table.json"
json_logs = json.load(open(path))
frame = pd.DataFrame(columns=json_logs["columns"], data=json_logs["data"])
frame.drop(columns=columns_to_drop, inplace=True)
frame.rename(columns=column_renaming, inplace=True)
frame = frame[columns_order]
frame["augmentations"] = frame["augmentations"].apply(lambda aug_batch: [aug.replace("<aug>", "").replace("</aug>", "") for aug in aug_batch[:4]] if len(aug_batch) > 0 else None)
frame

test_dict["BOSS_Sentiment_ID_BERT_ICR"] = Dataset.from_pandas(frame)
test_dict.push_to_hub("LLM-TTA-Augmentation-Logs")
test_dict

Deleting unused files from dataset repository: 100%|██████████| 1/1 [00:00<00:00,  3.25it/s]


DatasetDict({
    BOSS_Sentiment_ID_BERT_ICR: Dataset({
        features: ['outcome', 'original_text', 'augmentations', 'original_predicted_class', 'tta_predicted_class', 'label', 'tta_inference_latency', 'original_prediction_entropy', 'tta_prediction_entropy', 'prediction_entropy_decreased', 'prediction_entropy_decrease', 'tta_mean_class_probs', 'tta_all_class_probs', 'tta_all_class_entropy'],
        num_rows: 38904
    })
})

In [2]:
# create DatsetDict

combined_datset = DatasetDict()
for split_name in local_paths:
    local_path = "/home/kyobrien/repos/In-Context-Domain-Transfer-Improves-Out-of-Domain-Robustness/datasets/analysis/export_logs/export_run_analysis.ipynb"
    json_file = json.load(open(local_path))
    frame = pd.DataFrame(columns=json_file["columns"], data=json_file["data"])
    dataset = Dataset.from_pandas(frame)
    combined_datset[split_name] = dataset
    break

combined_datset


NameError: name 'local_paths' is not defined